# Gaia Kinematics

In [ ]:
import matplotlib.pyplot as plt

import numpy as np
import astropy.units as u

from astropy.coordinates import SkyCoord

from astropy.table import QTable
from astroquery.gaia import Gaia

# Motions of Objects

A big part of the Gaia dataset we have not looked at is the data on the motion of stars through space.  

<p>
    <img src="https://uwashington-astro300.github.io/A300_images/ProperMotion.png" width = "400">
</p>

Proper motion is the measure of the observed changes in the apparent places of stars, as seen from the center of mass of the Solar System, compared to the more distant stars. Knowledge of the proper motion, distance, and radial velocity allows calculations of an object's motion through space.

In the `gaia_source_lite` database, the proper motion data is found in the columns:

```
pmra                        Proper motion in right ascension direction
pmdec                       Proper motion in declination direction
radial_velocity             Radial velocity
```

### NGC 2682 (M 67) is a very well studied open star cluster in the northern skies

<p>
<img src="https://uwashington-astro300.github.io/A300_images/M67.jpg" width = "500">
</p>


- We used this cluster to make a CMD
- Let's take a look at the motion of the stars in the cluster

In [ ]:
target_coords = SkyCoord.from_name('NGC 2682')

In [ ]:
target_coords

In [ ]:
print(target_coords.get_constellation())

---

## Gaia query for 10σ data for our target

- Note the use of `f-string` and variables in the query
- Gaia wants the object coordinates in degrees
- We will use distances between 750 pc and 1000 pc for cluster stars

In [ ]:
my_query = f"""
SELECT TOP 1500
source_id, ra, dec, phot_g_mean_mag, bp_rp, parallax, parallax_over_error, pmra, pmdec, radial_velocity
FROM gaiadr3.gaia_source_lite
WHERE DISTANCE( POINT({target_coords.ra.degree}, {target_coords.dec.degree}), POINT(ra, dec) ) < 0.60
AND parallax_over_error > 10
AND parallax BETWEEN 1 AND 1.33
AND radial_velocity IS NOT NULL
ORDER BY parallax DESC
"""

In [ ]:
print(my_query)

In [ ]:
my_job_query = Gaia.launch_job_async(my_query)

In [ ]:
print(my_job_query)

### Query returned 530 objects. Less than the `SELECT` limit of 1500, so I am not cutting off objects

In [ ]:
my_table = my_job_query.get_results()

In [ ]:
my_table[0:2]

## Proper Motion of objects in NGC 2682 (M 67)

In [ ]:
fig, ax = plt.subplots(
    figsize = (8, 8), 
    constrained_layout = True
)

ax.set_ylim(-27,27)
ax.set_xlim(-27,27)

ax.tick_params(axis='both', which='major', labelsize = 18)

###

ax.set_xlabel("PM Ra",
              fontfamily = 'serif',
              fontsize = 25) 

ax.set_ylabel("PM Dec",
              fontfamily = 'serif',
              fontsize = 25)

### Plot Data ###

ax.vlines(0, -30, 30,
          color = 'Orchid',
          linewidth = 3,
          linestyle = '--')

ax.hlines(0, -30, 30,
          color = 'Orchid',
          linewidth = 3,
          linestyle = '--')

ax.plot(my_table['pmra'], my_table['pmdec'],
        color = "DarkOrchid",
        marker = "*",
        linestyle = "None",
        markersize = 10
       );

---
## Radial Velocity of objects in NGC 2682 (M 67)

In [ ]:
np.max(my_table['radial_velocity']), np.min(my_table['radial_velocity'])

In [ ]:
my_bins = np.arange(-35,120,2)
my_bins

In [ ]:
fig, ax = plt.subplots(
    figsize = (10, 5), 
    constrained_layout = True
)

ax.set_xlabel("Radial Velocity (km/s)",
              fontfamily = 'serif',
              fontsize = 15)

ax.set_ylabel("Number of Stars",
              fontfamily = 'serif',
              fontsize = 15)

ax.hist(my_table['radial_velocity'],
        bins = my_bins,
        histtype = 'stepfilled',
        facecolor = 'MediumOrchid');

---
## Total velocity through space of objects in NGC 2682 (M 67)

The total velocity through space of our objects can be calculated in a few steps:

### 1. Calculate the total proper motion ($\mu_{t}$): $\ \ \mu_{t}^2\ = \ \mu_{RA}^2 + \mu_{DEC}^2$

In [ ]:
my_table['Total_PM'] = np.sqrt(my_table['pmra'] ** 2 + my_table['pmdec'] ** 2)

In [ ]:
my_table[0:2]

### 2. Calculate the tangential velocity ($v_{tan}$): $\ \ v_{tan} \ = 4.74047 \frac{\mu_{tan}}{parallax}$

- 4.74047 is a conversion constant representing one AU/year in km/s.

In [ ]:
my_table['Tan_Velocity'] = 4.74047 * ( my_table['Total_PM'] / my_table['parallax'] )

In [ ]:
my_table[0:2]

### 3. Calculate the total space velocity ($v_{total}$): $\ \ v_{total}^2\ = \ v_{tan}^2 + v_{radial}^2$

In [ ]:
my_table['Space_Velocity'] = np.sqrt(my_table['Tan_Velocity'] ** 2 + my_table['radial_velocity'] ** 2)

In [ ]:
my_table[0:2]

## Plot it up

In [ ]:
np.max(my_table['Space_Velocity']), np.min(my_table['Space_Velocity'])

In [ ]:
my_bins = np.arange(0,130,2)
my_bins

In [ ]:
fig, ax = plt.subplots(
    figsize = (10, 5), 
    constrained_layout = True
)

ax.set_xlabel("Total Space Velocity (km/s)",
              fontfamily = 'serif',
              fontsize = 15)

ax.set_ylabel("Number of Stars",
              fontfamily = 'serif',
              fontsize = 15)

ax.hist(my_table['Space_Velocity'],
        bins = my_bins,
        histtype = 'stepfilled',
        facecolor = 'MediumOrchid');